In [ ]:
import brightway2 as bw
import bw2data as bd
import lxml.builder
import networkx as nx
from lxml.builder import ElementMaker
from lxml.etree import tostring
#import pygephi

### Load the required project and select the desired database

In [ ]:
bw.projects.set_current("alicia")
selected_database = bw.Database("fuelcell_exact202307")

### For checking the life cycle inventory

To print the all the details concerning the activities and corresponding exchanges from the selected database. Only use, *if required*

In [ ]:
from print_lci import *
print_lci_all (selected_database)

In [ ]:
from print_lci import *
print_lci_classify(selected_database) #classifies the activities within the selected database.

### Convert to graphml file.

Declare colors and other parameters required for the various nodes (activities, technosphere flows etc.) and edges/lines here.


In [ ]:
#Colors for nodes and edges
activity_node_color = "blue"
production_node_color = 'black'
production_edge_color = 'black'
technosphere_node_color = 'purple'
technosphere_edge_color = 'purple'
biosphere_node_color ='green'
biosphere_edge_color = 'green'

#line types for nodes and edges
production_line_style='Solid'
technosphere_line_style='Dash'
biosphere_line_style='Dots'

# node widths and heights
parent_activity_width = 26
production_node_width = 22
technosphere_node_width = 20
biosphere_node_width = 20

### Conversion of LCI to network diagram
Convert the activities, the products, technosphere and biosphere flows into an interconnected network diagram. This resultant file is stored in the graphml file format. The "draw" functions of networkx are used because graphml allows much more flexibility in terms of export and editing using open-source graphml editing programs such as gephi and cytoscape. One disadvantage of graphml is that it only allows datatypes of numbers (float, int etc.) and strings. Any other datatype as node or edge attributes will result in a "type" error. Another option could be use the gml format, i.e a format created by the team behind networkx - [GML format](https://networkx.org/documentation/stable/reference/readwrite/generated/networkx.readwrite.gml.write_gml.html).

In [ ]:
from bw_db_extract import *
graph = nx.DiGraph()
for activity in selected_database:

    # Iterate over each exchange in the activity
    for exchange in activity.exchanges():

        #Add the parent activity as a node. Repeated parent activities (as the loop iterates)  will be ignored by networkX
        activity_name = "Activity - " + activity['name']
        graph.add_node(activity_name, label=activity['name'], amount=0,unit=activity['unit'], location=activity['location'],type=activity['type'], from_database=activity['database'],  color=activity_node_color, width=parent_activity_width)
        
        # Production 'output' are added as outputs from the parent activity
        # Note that, within activities, all the exchanges are invoked using the available input, output tuples within each exchaneg dictionary. This way, the code redundacncies arising from the differences in the data formats of exchanges for type1 and type2 activities can be minimized.

        if exchange.as_dict()['type'] =='production':

            #extract the database name from the output tuple within the exchange dictionary
            name_production_database = exchange.as_dict()['output'][0]
            exchange_name = exchange.as_dict()['output'][1]
            exchange_type = exchange.as_dict()['type']

            #extract the required parameters from this database for production
            exchange_details = extract_details(name_production_database, exchange_name, exchange_type)
            production_database, production_display_name, production_reference_product, production_location, production_unit  = exchange_details [0:5]

            #add a node corresponding to the exchange
            graph.add_node(production_display_name, label=production_reference_product, amount=exchange.as_dict()['amount'], unit=production_unit, location=production_location, type=exchange.as_dict()['type'],from_database=name_production_database,  color=production_node_color, width = production_node_width)

            #add the arrow/edge connecting this exchange to the parent activity "node"
            graph.add_edge(activity_name, production_display_name, color=production_edge_color, line_type=production_line_style, arrows=True)

        elif exchange.as_dict()['type'] =='technosphere':

            name_technosphere_database = exchange.as_dict()['input'][0]
            exchange_name = exchange.as_dict()['input'][1]
            exchange_type = exchange.as_dict()['type']

            #extract the required parameters from this database for production
            exchange_details = extract_details(name_technosphere_database, exchange_name, exchange_type)
            technosphere_database, technosphere_display_name, technosphere_reference_product, technosphere_location, technosphere_unit  = exchange_details [0:5]

            #add a node corresponding to the exchange
            graph.add_node(technosphere_reference_product, label=technosphere_display_name, amount=exchange.as_dict()['amount'], unit=technosphere_unit, location=technosphere_location, type=exchange.as_dict()['type'],from_database=name_technosphere_database,  color=technosphere_node_color, width = technosphere_node_width)

            #add the arrow/edge connecting this exchange to the parent activity "node"
            graph.add_edge(technosphere_reference_product, activity_name, color=technosphere_edge_color,  line_type=technosphere_line_style, arrows=True)

        elif exchange.as_dict()['type'] =='biosphere':

            name_biosphere_database = exchange.as_dict()['input'][0]
            exchange_name = exchange.as_dict()['input'][1]
            exchange_type = exchange.as_dict()['type']

            #extract the required parameters from this database for production
            exchange_details = extract_details(name_biosphere_database, exchange_name, exchange_type)
            biosphere_database, biosphere_display_name, biosphere_categories, biosphere_unit = exchange_details [0:4]


            # the node and edges/arrows are added to the graph
            graph.add_node(biosphere_categories, label=biosphere_display_name, amount=exchange.as_dict()['amount'], unit=biosphere_unit, location=biosphere_categories, type=exchange.as_dict()['type'],from_database=name_biosphere_database,  color=biosphere_node_color, width = biosphere_node_width)

            graph.add_edge(activity_name,biosphere_categories, color=biosphere_edge_color, line_type=biosphere_line_style, arrows=True)

#Export to graphml format
nx.write_graphml(graph,'fuelcell_exact202307.graphml', prettyprint=True)

#just for debugging, we can make a live plot. Note: Do not use this for except for debugging because the figure is immutable.
#nx.draw(graph,  with_labels=True, node_shape='s', node_color='black', edge_color='blue')

In [ ]:
#Old code, for future deletion

graph = nx.DiGraph()

for activity in selected_database:


    #print(activity['name'], activity['unit'], activity['type'], activity['location']) # for debugging only

    # Iterate over each exchange
    for exchange in activity.exchanges():

        #Add the parent activity as a node. Repeated parent activities (as the loop iterates)  will be ignored by networkX
        
        graph.add_node(activity['name'], label="Activity - " + activity['name'], amount=0,unit=activity['unit'], location=activity['location'],type=activity['type'], from_database=activity['database'],  color=activity_node_color)
        
        #Production 'output' are added as outputs from the parent activity

        if exchange.as_dict()['type'] =='production':
            production_display_name = exchange.as_dict()["reference product"] #+ "-" + str(exchange.as_dict()["amount"]) + " " + exchange.as_dict()["unit"]
            
            #add a node corresponding to the exchange
            graph.add_node(production_display_name, label=exchange.as_dict()['reference product'], amount=exchange.as_dict()['amount'], unit=exchange.as_dict()['unit'], location=exchange.as_dict()['location'], type=exchange.as_dict()['type'],from_database=exchange.as_dict()['database'],  color=production_node_color)

            #add the arrow/edge connecting this exchange to the parent activity "node"
            graph.add_edge(activity['name'], production_display_name, color=production_edge_color, line_style=production_line_style, arrows=True)

        #Technosphere 'input' exchanges to the parent activity are added

        elif exchange.as_dict()['type'] =='technosphere' :

            #add a node corresponding to the exchange
            graph.add_node(exchange.as_dict()['reference product'], label=exchange.as_dict()['name'], amount=exchange.as_dict()['amount'], unit=exchange.as_dict()['unit'], location=exchange.as_dict()['location'], type=exchange.as_dict()['type'],from_database=exchange.as_dict()['database'],  color=technosphere_node_color)

            #add the arrow/edge connecting this exchange to the parent activity "node"
            graph.add_edge(exchange.as_dict()['reference product'], activity['name'], color=technosphere_edge_color,  line_style=technosphere_line_style, arrows=True)
            
        #Biosphere 'output' exchanges from the parent activity are added to the graph
        elif exchange.as_dict()['type'] =='biosphere':

            #Only the key, amount and type of the bioshpere exchange are present in the exchange.as_dict(). Therefore to get the, name of exchange, the key must be used to invoke the name from the cocnerned biosphere database. The key is present as a tuple of the form - 'input': ('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9').
            biodb=bw.Database("biosphere3")
            name_biosphere_exchange = biodb.get(exchange.as_dict()['input'][1])['name']
            unit_biosphere_exchange = biodb.get(exchange.as_dict()['input'][1])['unit']

            #biosphere category is a tuple in the form (air, urban air close to ground). Therefore, we store it as string before referring to it as label and location in the corresponding node.
            biosphere_category = biodb.get(exchange.as_dict()['input'][1])["categories"][0]
            biosphere_category_detail = biodb.get(exchange.as_dict()['input'][1])["categories"][0]

            # the node and edges/arrows are added to the graph
            graph.add_node(name_biosphere_exchange, label=biosphere_category, amount=exchange.as_dict()['amount'], unit=unit_biosphere_exchange, location=biosphere_category_detail, type=exchange.as_dict()['type'],from_database="biosphere",  color=biosphere_node_color)

            graph.add_edge(activity['name'],name_biosphere_exchange, color=biosphere_edge_color, line_style=biosphere_line_style, arrows=True)
        # Add edges to the graph connecting the activities
       # if exchange.as_dict()['type'] =='technosphere':
        #        

        #print('next activity')
# Export the graph to GEXF format, if requird
#nx.write_gexf(graph, "my_graph4.gexf")

#Export to graphml format
nx.write_graphml(graph,'EXACT_Loop3_20230327_rev1.graphml', prettyprint=True)

#just for debugging, we can make a live plot. Note: Do not use this for except for debugging because the figure is immutable.
nx.draw(graph,  with_labels=True, node_shape='s', node_color='black', edge_color='blue')